In [1]:
import rawpy
import cv2
import numpy as np

# Set resize scale factor
newWidth = 1000
newHeight = 1000
scaleFactor = 0.08
filterKernelSize = 3
img = 0

# Set Image path in array
pathRawImage = [
    "raw image/raw image 1.NEF",
    "raw image/raw image 2.NEF",
    "raw image/raw image 3.NEF",
    "raw image/raw image 4.NEF",
]

pathGTIGrayScaled = [
    "ground truth image/GTI Gray Scaled 1.tiff",
    "ground truth image/GTI Gray Scaled 2.tiff",
    "ground truth image/GTI Gray Scaled 3.tiff",
    "ground truth image/GTI Gray Scaled 4.tiff",
]

pathGTIRGBScaled = [
    "ground truth image/GTI Scaled 1.tiff",
    "ground truth image/GTI Scaled 2.tiff",
    "ground truth image/GTI Scaled 3.tiff",
    "ground truth image/GTI Scaled 4.tiff",
]

# Set Canny parameters
cannyParamSet = [
    {"tresh1": 25, "tresh2": 200, "apertureSize": 3, "L2gradient": True},
    {"tresh1": 100, "tresh2": 150, "apertureSize": 3, "L2gradient": True},
]

In [2]:
# Functions ***********************************************************************************************************


# Converter +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
class Converter:
    # Convert Raw to BGR
    def raw2bgr(imgSet, cameraWB=True):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(
                rawpy.imread(imgSet).postprocess(use_camera_wb=cameraWB),
                cv2.COLOR_RGB2BGR,
            )
        else:
            return [
                cv2.cvtColor(
                    rawpy.imread(img).postprocess(use_camera_wb=cameraWB),
                    cv2.COLOR_RGB2BGR,
                )
                for img in imgSet
            ]

    # Convert RGB to Gray
    def rgb2gray(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(imgSet, cv2.COLOR_RGB2GRAY)
        else:
            return [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgSet]

    # Convert RGB to BGR
    def rgb2bgr(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(imgSet, cv2.COLOR_RGB2BGR)
        else:
            return [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in imgSet]

    # Convert BGR to Gray
    def bgr2gray(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(imgSet, cv2.COLOR_BGR2GRAY)
        else:
            return [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgSet]

    # Convert BGR to RGB
    def bgr2rgb(imgSet):
        # Convert BGR image to RGB
        return [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in imgSet]

    # Convert Gray to BGR
    def gray2bgr(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(imgSet, cv2.COLOR_GRAY2BGR)
        else:
            return [cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) for img in imgSet]

    # Convert Gray to RGB
    def gray2rgb(imgSet):
        # Convert Gray image to RGB
        return [cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) for img in imgSet]


# Manipulate image ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
class Image:
    # Read image set
    def readSet(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.imread(imgSet)
        else:
            return [cv2.imread(img) for img in imgSet]

    # Save image to TIFF
    def saveSet(imgSet, fileName):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            cv2.imwrite(fileName, imgSet)
        elif len(imgSet) != len(fileName):
            print("Error: Image length mismatch")
        else:
            for i in range(len(imgSet)):
                cv2.imwrite(fileName[i], imgSet[i])
        return None

    # Scale images
    def scaleSetBy(imgSet, scaleFactor=0.1):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.resize(imgSet, None, fx=scaleFactor, fy=scaleFactor)
        else:
            return [
                cv2.resize(img, None, fx=scaleFactor, fy=scaleFactor) for img in imgSet
            ]

    def scaleSetToHeight(imgSet, height):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.resize(
                imgSet, (int(imgSet.shape[1] * height / imgSet.shape[0]), height)
            )
        else:
            return [
                cv2.resize(img, (int(img.shape[1] * height / img.shape[0]), height))
                for img in imgSet
            ]

    def scaleSetToWidth(imgSet, width):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.resize(
                imgSet, (width, int(imgSet.shape[0] * width / imgSet.shape[1]))
            )
        else:
            return [
                cv2.resize(img, (width, int(img.shape[0] * width / img.shape[1])))
                for img in imgSet
            ]

    # Concatenate images
    def concat(img1, img2, axis=1):
        # Concatenate two images, axis=1 for horizontal, axis=0 for vertical
        # Get new height and width
        newHeight = max(img1.shape[0], img2.shape[0])
        newWidth = max(img1.shape[1], img2.shape[1])
        # Concatenate two images with flexible size
        return np.concatenate(
            [
                cv2.copyMakeBorder(
                    img,
                    0,
                    newHeight - img.shape[0],
                    0,
                    newWidth - img.shape[1],
                    cv2.BORDER_CONSTANT,
                )
                for img in [img1, img2]
            ],
            axis=axis,
        )

    def concatSet(imgSet, axis=1):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            print("Error: Image set is not array")
            return None
        elif len(imgSet) == 0:
            print("Error: Image set is empty")
            return None
        else:
            # Loop through all images, get max height and width
            maxHeight = max([img.shape[0] for img in imgSet])
            maxWidth = max([img.shape[1] for img in imgSet])
            # Concatenate all images with various size in set
            return np.concatenate(
                [
                    cv2.copyMakeBorder(
                        img,
                        0,
                        maxHeight - img.shape[0],
                        0,
                        maxWidth - img.shape[1],
                        cv2.BORDER_CONSTANT,
                    )
                    for img in imgSet
                ],
                axis=axis,
            )

    # Show all images
    def showSet(imgSet, name="image"):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            cv2.imshow(name, imgSet)
            return None
        elif len(imgSet) == 0:
            print("Error: Image set is empty")
            return None
        else:
            for i in range(len(imgSet)):
                fileName = pathRawImage[i].split("/")[-1]
                cv2.imshow(name + str(i + 1) + ': "' + fileName + '"', imgSet[i])


# Smooth Filter +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# 1. Average Filter
# 2. Box Filter
# 3. Gaussian Filter
# 4. Median Filter
# 5. Bilateral Filter
# 6. Non-Local Means Filter
# 7. Custom Kernel Filter
class SmoothFilter:

    def applyAverageFilter(imgSet, kernelSizeX=5, kernelSizeY=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.blur(imgSet, (kernelSizeX, kernelSizeY))
        else:
            return [cv2.blur(img, (kernelSizeX, kernelSizeY)) for img in imgSet]

    def applyBoxFilter(imgSet, kernelSizeX=5, kernelSizeY=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.boxFilter(imgSet, -1, (kernelSizeX, kernelSizeY))
        else:
            return [
                cv2.boxFilter(img, -1, (kernelSizeX, kernelSizeY)) for img in imgSet
            ]

    def applyGaussianBlurFilter(imgSet, kernelSizeX=5, kernelSizeY=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.GaussianBlur(imgSet, (kernelSizeX, kernelSizeY), 0)
        else:
            return [
                cv2.GaussianBlur(img, (kernelSizeX, kernelSizeY), 0) for img in imgSet
            ]

    def applyMedianBlurFilter(imgSet, kernelSize=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.medianBlur(imgSet, kernelSize)
        else:
            return [cv2.medianBlur(img, kernelSize) for img in imgSet]

    def applyBilateralFilter(imgSet, d=9, sigmaColor=75, sigmaSpace=75):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.bilateralFilter(imgSet, d, sigmaColor, sigmaSpace)
        else:
            return [
                cv2.bilateralFilter(img, d, sigmaColor, sigmaSpace) for img in imgSet
            ]

    def applyNonLocalMeansFilter(imgSet, h=10, searchWindowSize=20):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.fastNlMeansDenoising(imgSet, None, h, searchWindowSize)
        else:
            return [
                cv2.fastNlMeansDenoising(img, None, h, searchWindowSize)
                for img in imgSet
            ]

    def applyCustomKernelFilter(imgSet, kSize):
        kernel = np.ones((kSize, kSize), np.float32) / (kSize * kSize)
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.filter2D(imgSet, -1, kernel)
        else:
            return [cv2.filter2D(img, -1, kernel) for img in imgSet]


# Edge Detection Filter ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# 1. Canny Edge Detection
# 2. Sobel Edge Detection
# 3. Laplacian Edge Detection
# 4. Scharr Edge Detection
# 5. Prewitt Edge Detection
class EdgeFilter:
    def applyCanny(imgSet, CannyParam):
        # Threshold 1 and 2 are used to detect strong and weak edges(minVal = 0, maxVal = 255)
        # ApertureSize is the size of Sobel kernel used to find image gradients
        # L2gradient is a flag to specify the equation for finding gradient magnitude
        # Check if image is array or not

        cannySet = []
        if not isinstance(imgSet, list):
            for j in range(len(CannyParam)):
                cannySet.append(
                    cv2.Canny(
                        imgSet,
                        CannyParam[j]["tresh1"],
                        CannyParam[j]["tresh2"],
                        apertureSize=CannyParam[j]["apertureSize"],
                        L2gradient=CannyParam[j]["L2gradient"],
                    )
                )
        else:
            for i in range(len(imgSet)):
                cannySet.append([])
                for j in range(len(CannyParam)):
                    cannySet[i].append(
                        cv2.Canny(
                            imgSet[i],
                            CannyParam[j]["tresh1"],
                            CannyParam[j]["tresh2"],
                            apertureSize=CannyParam[j]["apertureSize"],
                            L2gradient=CannyParam[j]["L2gradient"],
                        )
                    )

        return cannySet


class Compute:
    def peakSignalToNoiseRatio(img1, img2, decimal=4):
        # Check if image is array or not
        if not isinstance(img1, list) and not isinstance(img2, list):
            return cv2.PSNR(img1, img2)
        elif not isinstance(img1, list) or not isinstance(img2, list):
            print("Error: Image type mismatch")
        elif len(img1) != len(img2):
            print("Error: Image length mismatch")
        else:
            # Compute PSNR
            psnr = [None] * len(img1)
            for i in range(len(img1)):
                psnr[i] = round(cv2.PSNR(img1[i], img2[i]), decimal)

            return psnr
        return None

    def psnrToString(psnr, imgSetA, imgSetB):
        # Check if PSNR is array or not
        if not isinstance(psnr, list):
            print("PSNR: " + "imgSetA" + " | " + "imgSetB" + " = " + str(psnr))
        else:
            for i in range(len(psnr)):
                print(
                    "PSNR: "
                    + imgSetA[i].split("/")[-1]
                    + " | "
                    + imgSetB[i].split("/")[-1]
                    + " = "
                    + str(psnr[i])
                )

    def compressionRatio(img1, img2, decimal=4):
        # Check if image is array or not
        if not isinstance(img1, list) and not isinstance(img2, list):
            return round((img1.nbytes / img2.nbytes), decimal)
        elif not isinstance(img1, list) or not isinstance(img2, list):
            print("Error: Image type mismatch")
        elif len(img1) != len(img2):
            print("Error: Image length mismatch")
        else:
            # Compute Compression Ratio
            cr = [None] * len(img1)
            for i in range(len(img1)):
                cr[i] = round((img1[i].nbytes / img2[i].nbytes), decimal)

            return cr
        return None

    def crToString(cr, imgSetA, imgSetB):
        # Check if CR is array or not
        if not isinstance(cr, list):
            print("CR: " + "imgSetA" + " | " + "imgSetB" + " = " + str(cr))
        else:
            for i in range(len(cr)):
                print(
                    "CR: "
                    + imgSetA[i].split("/")[-1]
                    + " | "
                    + imgSetB[i].split("/")[-1]
                    + " = "
                    + str(cr[i])
                )

In [3]:
# Step 1: Reading RAW Image and converting to BGR for OpenCV (Read RAW Image) ===============================# Get Processed Image
rawSet = Converter.raw2bgr(pathRawImage)
rawScaledImageSet = Image.scaleSetBy(rawSet, scaleFactor)

# Step 2: Pre-process image, create "Ground Truth Image" (Grayscale, Resize, Save) ==========================
raw2GraySet = Converter.bgr2gray(rawSet)
scaledRaw2GraySet = Image.scaleSetBy(raw2GraySet, scaleFactor)
scaledRawSet = Image.scaleSetBy(rawSet, scaleFactor)

# Save the processed image object to tagged image file TIFF as ground truth image
Image.saveSet(scaledRaw2GraySet, pathGTIGrayScaled)
Image.saveSet(scaledRawSet, pathGTIRGBScaled)

# Read the saved ground truth image
gtiGraySet = Image.readSet(pathGTIGrayScaled)
gtiRGBSet = Image.readSet(pathGTIRGBScaled)

# Prepare the image
gtiGraySet = Converter.rgb2gray(gtiGraySet)

In [4]:
# Step 3: Apply blur filter to the image (Smoothing Filter) =================================================
# 1. Average filter
sfAverageGraySet = SmoothFilter.applyAverageFilter(
    gtiGraySet, filterKernelSize, filterKernelSize
)
sfAverageRGBSet = SmoothFilter.applyAverageFilter(
    gtiRGBSet, filterKernelSize, filterKernelSize
)

# 2. Box filter
sfBoxGraySet = SmoothFilter.applyBoxFilter(
    gtiGraySet, filterKernelSize, filterKernelSize
)
sfBoxRGBSet = SmoothFilter.applyBoxFilter(gtiRGBSet, filterKernelSize, filterKernelSize)

# 3. Gaussian filter
sfGaussianGraySet = SmoothFilter.applyGaussianBlurFilter(
    gtiGraySet, filterKernelSize, filterKernelSize
)
sfGaussianRGBSet = SmoothFilter.applyGaussianBlurFilter(
    gtiRGBSet, filterKernelSize, filterKernelSize
)

# 4. Median filter
sfMedianGraySet = SmoothFilter.applyMedianBlurFilter(gtiGraySet, filterKernelSize)
sfMedianRGBSet = SmoothFilter.applyMedianBlurFilter(gtiRGBSet, filterKernelSize)

# 5. Bilateral filter
sfBilateralGraySet = SmoothFilter.applyBilateralFilter(gtiGraySet, 9, 75, 75)
sfBilateralRGBSet = SmoothFilter.applyBilateralFilter(gtiRGBSet, 9, 75, 75)

# 6. Non-local means filter
sfNlMeanGraySet = SmoothFilter.applyNonLocalMeansFilter(gtiGraySet, 10, 20)
sfNlMeanRGBSet = SmoothFilter.applyNonLocalMeansFilter(gtiRGBSet, 10, 20)

# 7. Custom kernel filter
sfCustKernelGraySet = SmoothFilter.applyCustomKernelFilter(gtiGraySet, filterKernelSize)
sfCustKernelRGBSet = SmoothFilter.applyCustomKernelFilter(gtiRGBSet, filterKernelSize)

# Compile all filtered set
sfFullGraySet = [
    sfAverageGraySet,  # sf1
    sfBoxGraySet,  # sf2
    sfGaussianGraySet,  # sf3
    sfMedianGraySet,  # sf4
    sfBilateralGraySet,  # sf5
    sfNlMeanGraySet,  # sf6
    sfCustKernelGraySet,  # sf7
]
sfFullRGBSet = [
    sfAverageRGBSet,  # sf1
    sfBoxRGBSet,  # sf2
    sfGaussianRGBSet,  # sf3
    sfMedianRGBSet,  # sf4
    sfBilateralRGBSet,  # sf5
    sfNlMeanRGBSet,  # sf6
    sfCustKernelRGBSet,  # sf7
]

In [5]:
# Step 4: Apply edge detection filter to the image (Edge Detection Filter) ===================================
# 1. Canny edge detection | sf1CannySet[i][j] = i-th image, j-th Canny parameter
# A. Without smoothing
cannyGraySet = EdgeFilter.applyCanny(gtiGraySet, cannyParamSet)  # Without smoothing
cannyRGBSet = EdgeFilter.applyCanny(gtiRGBSet, cannyParamSet)  # Without smoothing

# B. Average filter
sf1CannyGraySet = EdgeFilter.applyCanny(sfAverageGraySet, cannyParamSet)
sf1CannyRGBSet = EdgeFilter.applyCanny(sfAverageRGBSet, cannyParamSet)

# C. Box filter
sf2CannyGraySet = EdgeFilter.applyCanny(sfBoxGraySet, cannyParamSet)
sf2CannyRGBSet = EdgeFilter.applyCanny(sfBoxRGBSet, cannyParamSet)

# D. Gaussian filter
sf3CannyGraySet = EdgeFilter.applyCanny(sfGaussianGraySet, cannyParamSet)
sf3CannyRGBSet = EdgeFilter.applyCanny(sfGaussianRGBSet, cannyParamSet)

# E. Median filter
sf4CannyGraySet = EdgeFilter.applyCanny(sfMedianGraySet, cannyParamSet)
sf4CannyRGBSet = EdgeFilter.applyCanny(sfMedianRGBSet, cannyParamSet)

# F. Bilateral filter
sf5CannyGraySet = EdgeFilter.applyCanny(sfBilateralGraySet, cannyParamSet)
sf5CannyRGBSet = EdgeFilter.applyCanny(sfBilateralRGBSet, cannyParamSet)

# G. Non-local means filter
sf6CannyGraySet = EdgeFilter.applyCanny(sfNlMeanGraySet, cannyParamSet)
sf6CannyRGBSet = EdgeFilter.applyCanny(sfNlMeanRGBSet, cannyParamSet)

# H. Custom kernel filter
sf7CannyGraySet = EdgeFilter.applyCanny(sfCustKernelGraySet, cannyParamSet)
sf7CannyRGBSet = EdgeFilter.applyCanny(sfCustKernelRGBSet, cannyParamSet)

In [6]:
# Declare image set, gray and rgb
set1 = cannyGraySet[img]
set2 = cannyRGBSet[img]
set3 = sf1CannyGraySet[img]
set4 = sf1CannyRGBSet[img]

# Convert gray channel to 3 channel
set2 = Converter.gray2bgr(set2)
set4 = Converter.gray2bgr(set4)

# Add reference image to the set
set1.insert(0, gtiGraySet[img])
set2.insert(0, gtiRGBSet[img])
set3.insert(0, sfAverageGraySet[img])
set4.insert(0, sfAverageRGBSet[img])

# Convert all to 3 channel
set1 = Converter.gray2rgb(set1)
set3 = Converter.gray2rgb(set3)

set1 = Image.concatSet(set1, axis=1)
set2 = Image.concatSet(set2, axis=1)
set3 = Image.concatSet(set3, axis=1)
set4 = Image.concatSet(set4, axis=1)

sideBySideA = Image.concat(set1, set3, axis=0)
sideBySideB = Image.concat(set2, set4, axis=0)
sideBySide = Image.concat(sideBySideA, sideBySideB, axis=0)

Image.showSet(sideBySide, "Filter")

In [7]:
# Step 4 Compute Peak Signal-to-Noise Ratio (PSNR), compression ratio =======================================

# psnr = [
#     Compute.peakSignalToNoiseRatio(groundTruthImageSet, imgSet, 2)
#     for imgSet in sfFullSet
# ]

# cr = [Compute.compressionRatio(groundTruthImageSet, imgSet, 2) for imgSet in sfFullSet]

In [8]:
# set1 = scaleImages(groundTruthImageSet, scaleFactor)
# set2 = scaleImages(Converter.gray2bgr(groundTruthImageSet), scaleFactor)
# rawXgroundTruthSet = concatImages(set1, set2)
# gtiXfilteredSet = concatenate(groundTruthImageSet, smoothFilteredSet[2])

# Image.showImages(rawImageSet)
# Image.showImages(groundTruthImageSet)
# Image.showImages(grayImageSet)
# Image.showImages(grayResizedImagesSet)
# Image.showImages(rawXgroundTruthSet, "Raw Image X Ground Truth Image")

# Smooth Filtered
# Image.show(smoothFilteredSet[0], "Smooth Filtered Image")  # Average Filtered Image
# Image.show(smoothFilteredSet[1], "Smooth Filtered Image") # Box Filtered Image
# Image.show(smoothFilteredSet[2], "Smooth Filtered Image") # Gaussian Filtered Image
# Image.show(smoothFilteredSet[3], "Smooth Filtered Image") # Median Filtered Image
# Image.show(smoothFilteredSet[4], "Smooth Filtered Image") # Bilateral Filtered Image
# Image.show(smoothFilteredSet[5], "Smooth Filtered Image") # Non-Local Means Filtered Image
# Image.show(smoothFilteredSet[6], "Smooth Filtered Image") # Custom Kernel Filtered Image

# Edge Detected Filtered
# Image.show(edgeDetectedSet, "Edge Detected Filtered Image")

# Concatenated
# Image.show(gtiXfilteredSet, "Average Filtered Image")

cv2.waitKey(0)
cv2.destroyAllWindows()